In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib



## Gradient descent

Forest analogy: You're lost in the mountain and has to find your way down

![](https://upload.wikimedia.org/wikipedia/commons/c/c7/Okanogan-Wenatchee_National_Forest%2C_morning_fog_shrouds_trees_%2837171636495%29.jpg)

## Gradient descent

* Gradient descent is an iterative optimization algorithm for finding the minimum of a function.

Math analogy: You have a function, and the only thing you know is the *direction* it moves

In multiple dimensions the *direction* is in multiple dimensions. So we don't just need the derivative of the function in one dimension, we need it in multiple dimensions! 

We need the gradients! That's why it's called gradient descent.

![](https://upload.wikimedia.org/wikipedia/commons/f/ff/Gradient_descent.svg)

## Gradients

A function $f$ can require a number of inputs.
So how do we find the gradient for one of the inputs? If we keep the others constant!



$\nabla f(a) = \left(\frac{\partial f}{\partial x_1}(a), \ldots, \frac{\partial f}{\partial x_n}(a)\right)$

In one point $a$, this gives us a list of partial derivatives, that tells us which direction each dimension is moving, in that exact point $a$.

This is called a **gradient**.

## The gradient descent algorithm

Let's revisit the science/suicide dataset:


In [ ]:
import pandas as pd
df = pd.read_csv('science.csv')

In [ ]:
df.plot.scatter(x=1, y=2)

We're still going for a *linear* relationship:

$$f(x) = \alpha x + \beta$$

And our loss function is still:

$$l = {{\sum^N_{i=1}(y_i - \hat{y_i})^2} \over N}$$

Which is the same as:

$$l = {{1 \over N} \sum^N_{i=1}(y_i - (\alpha x_i + \beta))^2}$$

We now know what to optimise! We have a loss function with two parameters: $\alpha$ and $\beta$.


$$l = {{1 \over N} \sum^N_{i=1}(y_i - (\alpha x_i + \beta))^2}$$

So we have to find the **partial** derivative for $\alpha$:

$${\partial l \over \partial \alpha} = {{1 \over N} \sum^N_{i=1} -2x_i(y_i - (\alpha x_i + \beta))}$$

We now know what to optimise! We have a loss function with two parameters: $\alpha$ and $\beta$.


$$l = {{1 \over N} \sum^N_{i=1}(y_i - (\alpha x_i + \beta))^2}$$

So we have to find the **partial** derivative for $\beta$:

$${\partial l \over \partial \beta} = {{1 \over N} \sum^N_{i=1} -2(y_i - (\alpha x_i + \beta))}$$

To start gradient descent, let's just pretend that $\alpha$ and $\beta$ are set to 0.

Whenever vi find a gradient, we get a **direction** at that specific point.
* If the gradient is positive, the function grows
* If the gradient is negative, the function declines

... Either is actuall bad, because we want the function to stay at zero!
So we will always move **in the opposite direction** of the gradient.

We can now update our parameters:

$$\alpha \leftarrow \alpha - {\partial l \over \partial \alpha} $$

$$\beta \leftarrow \beta - {\partial l \over \partial \beta} $$

## Learning rates

Gradient descent has a problem: local minima.

We can add a *learning rate* that scales the amount we are learning (between 0 and 1).

The learning rate will prevent us from 'jumping' too far.

We can now update our parameters with a learning rate $\gamma$:

$$\alpha \leftarrow \alpha - \gamma {\partial l \over \partial \alpha} $$

$$\beta \leftarrow \beta - \gamma {\partial l \over \partial \beta} $$

## Gradient descent in Python

In [ ]:
def loss(spendings, suimycides, a, b):
    N = len(spendings)
    total_error = 0.0
    for i in range(N):
        total_error += (suicides[i] - (a*spendings[i] + b))**2
    return total_error / N

In [ ]:
def update_a_and_b(spendings, suicides, a, b, gamma):
    dr_da = 0.0
    dr_db = 0.0
    N = len(spendings)

    for i in range(N):
        dr_da += -2 * spendings[i] * (suicides[i] - (a * spendings[i] + b))
        dr_db += -2 * (suicides[i] - (a * spendings[i] + b))

    # update a and b
    a = a - (dr_da/float(N)) * gamma
    b = b - (dr_db/float(N)) * gamma

    return a, b

In [ ]:
def train(spendings, suicides, a, b, gamma, epochs):
    image_counter = 2
    for e in range(epochs):
        a, b = update_a_and_b(spendings, suicides, a, b, gamma)

        if (e % 10 == 0):
            # log the progress
            print("epoch: ", str(e), "loss: "+str(loss(spendings, suicides, a, b)))
            print("a, b: ", a, b)
            plt.figure(image_counter)
            axes = plt.gca()
            plt.scatter(spendings, suicides)
            plt.plot(spendings, spendings*a + b)
            image_counter += 1
            
        if np.isnan(a):
            raise ValueError('Infinite error!')

In [ ]:
train(df['US science spending'], df['Suicides'], 0, 0, 0.1, 10000)

## Scaling data

Gradient descent is sensitive to big numbers:

* Large gradients = large movements = large losses = large gradients = ...

Solution: scale the data so it centers around 0

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
data = np.arange(0, 1000)

In [ ]:
StandardScaler().fit(data).transform(data)

In [ ]:
StandardScaler().fit(data.reshape(-1, 1)).transform(data.reshape(-1, 1))

In [ ]:
StandardScaler().fit_transform(data.reshape(-1, 1))

In [ ]:
scaled_spending = StandardScaler().fit_transform(df['US science spending'].values.reshape(-1, 1))
scaled_suicides = StandardScaler().fit_transform(df['Suicides'].values.reshape(-1, 1))

In [ ]:
train(scaled_spending, scaled_suicides, 0, 0, 0.01, 200)

## Gradient descent in `sklearn`

## Gradient descent: summary

* We have a loss function for our models
  * We like our loss to be **small**
* That loss function can be in multiple dimensions
  * It's **hard** to predict where the loss function is small
* Gradients gives us an idea on the *direction* the function is going
  * Direction **small** is good because it means a small loss

* Gradient descent steps in the direction of the loss
  * Until you find the smallest point

In [ ]:
from sklearn.linear_model import SGDRegressor
SGDRegressor?

## Gradient descent example in sklearn

In [ ]:
import pandas as pd
data = pd.read_csv("science.csv")
data

In [ ]:
xs = np.array(data['US science spending']).reshape(-1, 1)
ys = np.array(data['Suicides']).reshape(-1, 1)

In [ ]:
data.plot.scatter(x = 1, y = 2)

In [ ]:
model = SGDRegressor()
model.fit(xs, ys)

In [ ]:
model.predict([[100], [10000]])

In [ ]:
data.plot.scatter(x = 1, y = 2)
plt.plot(xs, model.predict(xs))

## Scaling data

Gradient descent goes in the direction of the gradient with respect to x.
If that gradient is very large, the steps we take are laaaarge.

What can we do to fix that?
Scale the data to be smaller!

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
xs_scaled = StandardScaler().fit_transform(xs)
xs_scaled

In [ ]:
model = SGDRegressor()
model.fit(xs_scaled, ys)

In [ ]:
plt.plot(xs_scaled, ys)
plt.plot(xs_scaled, model.predict(xs_scaled))

## Why don't we get any good results?!

To get the optimal solution, we have to take many steps towards the correct solution. 

## Exercise: 

* Use the `SGDRegressor` to fit a model
* Figure out how many steps the model is taking
* Increase that step to, say, 1000 and see whether your prediction is better
* What happens if you increase the steps to 10000?

## Recap

* Derivatives
  * The **rate of growth** for a function $f$
  * Normally defined at points like $x$: $f'(x)$
* Partial derivatives
  * Derivatives in multiple dimensions

* Gradient
  * A vector of derivatives, one for each dimension in a function $f$
* Gradient descent
  * A way to optimise a function by moving towards an optimum
  * For instance minimising a loss function